In [1]:
import math
import numpy as np
import os
from einops import rearrange
from MDAnalysis.analysis.dssp import DSSP 
import MDAnalysis as mda

# Remplace par le chemin que tu veux
os.chdir("/home/gcluzeau/Zone_de_travail/projet_court")

# Vérifie que ça a marché
print("Répertoire courant :", os.getcwd())

/home/gcluzeau/miniconda3/envs/env_proj_court/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Répertoire courant : /home/gcluzeau/Zone_de_travail/projet_court


Detection des structure secondaire 

In [ ]:
def parse_pdb_residues(pdb_path):
    """
    Parse residues from a PDB file, extracting atom coordinates for N, CA, C, O, H, and H1 atoms.
    Returns a dictionary of residues with atom coordinates.
    """
    residues = {}
    with open(pdb_path, 'r') as f:
        for line in f:
            if not line.startswith("ATOM"):
                continue

            # Extract relevant fields from fixed-width PDB format
            atom_name = line[12:16].strip()
            resname = line[17:20].strip()
            chain_id = line[21].strip()
            resseq = int(line[22:26])

            x = float(line[30:38])
            y = float(line[38:46])
            z = float(line[46:54])

            key = (chain_id, resseq)
            if key not in residues:
                residues[key] = {'resname': resname}

            # Store only backbone atoms and possible hydrogens
            if atom_name in {'N', 'CA', 'C', 'O', 'H', 'H1'}:
                residues[key][atom_name] = [x, y, z]

    return residues


def _get_hydrogen_atom_position(residue_keys, j, residues_dict):
    """
    Estimate the hydrogen (H) atom position of residue j using neighboring atom vectors.
    Returns None if required atoms are missing.
    """
    if j == 0:
        return None

    prev_key = residue_keys[j - 1]
    curr_key = residue_keys[j]

    res_prev = residues_dict.get(prev_key, {})
    res_curr = residues_dict.get(curr_key, {})

    N = res_curr.get('N')
    CA = res_curr.get('CA')
    C_prev = res_prev.get('C')

    if None in (N, CA, C_prev):
        return None

    # Calculate normalized vectors and estimate H position
    N = np.array(N)
    CA = np.array(CA)
    C_prev = np.array(C_prev)

    vec_cn = N - C_prev
    vec_cn /= np.linalg.norm(vec_cn)

    vec_can = N - CA
    vec_can /= np.linalg.norm(vec_can)

    vec_nh = vec_cn + vec_can
    vec_nh /= np.linalg.norm(vec_nh)

    return (N + 1.01 * vec_nh).tolist()


def distance_atom(atom1, atom2):
    """
    Compute Euclidean distance between two atoms, return inf if either is missing.
    """
    if atom1 is None or atom2 is None:
        return float('inf')
    return np.linalg.norm(np.array(atom1) - np.array(atom2))


def is_hydrogen_bond(res_1, res_2, j, residue_keys, residues_dict):
    """
    Determine if a hydrogen bond exists between two residues using an electrostatic energy approximation.
    """
    # Try to get hydrogen atom position (explicit or estimated)
    h_atom_coords = res_2.get('H') or res_2.get('H1')
    if h_atom_coords is None:
        h_atom_coords = _get_hydrogen_atom_position(residue_keys, j, residues_dict)
        if h_atom_coords is None:
            return False

    # Compute distances needed for electrostatic energy calculation
    d_oh = distance_atom(res_1.get('O'), h_atom_coords)
    d_ch = distance_atom(res_1.get('C'), h_atom_coords)
    d_on = distance_atom(res_1.get('O'), res_2.get('N'))
    d_cn = distance_atom(res_1.get('C'), res_2.get('N'))

    if float('inf') in {d_on, d_oh, d_cn, d_ch}:
        return False

    # Electrostatic energy approximation
    q1 = 0.42
    q2 = 0.20
    f = 332
    Emin = -0.5
    E_elec = q1 * q2 * f * (1 / d_on + 1 / d_ch - 1 / d_oh - 1 / d_cn)

    return E_elec < Emin


def find_beta_bridges(residues_dict):
    """
    Identify beta-sheet hydrogen bonding patterns and classify them as parallel or antiparallel.
    """
    residue_keys = sorted(residues_dict.keys())
    bridges = {}
    n = len(residue_keys)

    for i in range(n - 1):
        for j in range(i + 2, n):  # Avoid neighbors (i+1)
            res_i = residues_dict[residue_keys[i]]
            res_j = residues_dict[residue_keys[j]]

            # Check for antiparallel bridge
            if is_hydrogen_bond(res_i, res_j, j, residue_keys, residues_dict) and \
               is_hydrogen_bond(res_j, res_i, i, residue_keys, residues_dict):
                bridges[(residue_keys[i], residue_keys[j])] = 'antiparallel'

            # Check for parallel bridge
            elif j + 1 < n and i + 1 < n:
                res_jp1 = residues_dict[residue_keys[j + 1]]
                res_ip1 = residues_dict[residue_keys[i + 1]]
                if is_hydrogen_bond(res_i, res_jp1, j + 1, residue_keys, residues_dict) and \
                   is_hydrogen_bond(res_j, res_ip1, i + 1, residue_keys, residues_dict):
                    bridges[(residue_keys[i], residue_keys[j])] = 'parallel'
    return bridges


def annotate_beta_strands_on_sequence(residues_dict, bridges):
    """
    Assign 'E' to residues involved in beta strands based on detected bridges.
    """
    residue_keys = sorted(residues_dict.keys())
    n = len(residue_keys)
    ss_seq = ['-' for _ in residue_keys]

    for (res1, res2), _ in bridges.items():
        if res1 in residue_keys:
            ss_seq[residue_keys.index(res1)] = 'E'
        if res2 in residue_keys:
            ss_seq[residue_keys.index(res2)] = 'E'

    # Fill in gaps within E regions (extend strands)
    for i in range(1, n - 1):
        if ss_seq[i - 1] == 'E' and ss_seq[i + 1] == 'E' and ss_seq[i] == '-':
            ss_seq[i] = 'E'

    # Remove isolated 'E's
    for i in range(n):
        if ss_seq[i] == 'E':
            prev_is_E = (i > 0 and ss_seq[i - 1] == 'E')
            next_is_E = (i < n - 1 and ss_seq[i + 1] == 'E')
            if not prev_is_E and not next_is_E:
                ss_seq[i] = '-'

    return list(zip(residue_keys, ss_seq))


def assign(residues_dict):
    """
    Detect helix types (3-10, alpha, pi) using hydrogen bond patterns (i to i+3, +4, +5).
    Returns a one-hot encoded matrix for loop, helix, and strand.
    """
    residue_keys = sorted(residues_dict.keys(), key=lambda x: (x[0], x[1]))
    residues = [residues_dict[k] for k in residue_keys]
    L = len(residues)
    hbmap = np.zeros((L, L), dtype=bool)

    # Fill hydrogen bond map
    for i in range(L):
        for j in range(L):
            if i == j:
                continue
            if is_hydrogen_bond(residues[i], residues[j], j, residue_keys, residues_dict):
                hbmap[i, j] = True

    def pad(arr, pad_width):
        return np.pad(arr, pad_width, mode='constant', constant_values=0)

    # Detect helices from diagonal HB patterns
    turn3 = np.diagonal(hbmap, offset=3) > 0
    turn4 = np.diagonal(hbmap, offset=4) > 0
    turn5 = np.diagonal(hbmap, offset=5) > 0

    h3 = pad(turn3[:-1] * turn3[1:], (1, 3))
    h4 = pad(turn4[:-1] * turn4[1:], (1, 4))
    h5 = pad(turn5[:-1] * turn5[1:], (1, 5))

    helix4 = h4 + np.roll(h4, 1) + np.roll(h4, 2) + np.roll(h4, 3)
    h3 = h3 * ~np.roll(helix4, -1) * ~helix4
    h5 = h5 * ~np.roll(helix4, -1) * ~helix4

    helix3 = h3 + np.roll(h3, 1) + np.roll(h3, 2)
    helix5 = h5 + np.roll(h5, 1) + np.roll(h5, 2) + np.roll(h5, 3) + np.roll(h5, 4)

    helix = (helix3 + helix4 + helix5) > 0
    strand = np.zeros_like(helix, dtype=bool)
    loop = (~helix) & (~strand)

    onehot = np.stack([loop, helix, strand], axis=-1)
    return onehot, residue_keys


def combine_secondary_structure(residues_dict):
    """
    Combine helix and beta strand assignments into a final label per residue.
    Returns labeled residues and detected beta bridges.
    """
    onehot_ss, residue_keys = assign(residues_dict)
    bridges = find_beta_bridges(residues_dict)
    annotated = dict(annotate_beta_strands_on_sequence(residues_dict, bridges))

    combined = []
    for key, ss in zip(residue_keys, onehot_ss):
        label = 'L'  # Default: loop
        if ss[1]: label = 'H'  # Helix
        if annotated.get(key) == 'E': label = 'E'  # Strand
        combined.append(f"{key[0]}{key[1]}: {label}")
    return combined, bridges


def print_beta_bridges(bridges):
    """
    Print detected beta bridges with classification (parallel or antiparallel).
    """
    print("\nHydrogen bonds (detected beta bridges):")
    for (res1, res2), btype in bridges.items():
        print(f"  {btype.capitalize()} between {res1} and {res2}")



Main

In [3]:
if __name__ == "__main__":
    pdb_path = "1bl8.pdb"
    residues = parse_pdb_residues(pdb_path)
    result, bridges = combine_secondary_structure(residues)

    print_beta_bridges(bridges)
    print("\nFinal secondary structure:")
    for line in result:
        print(line)


Hydrogen bonds (detected beta bridges):

Final secondary structure:
A23: L
A24: L
A25: L
A26: L
A27: H
A28: H
A29: H
A30: H
A31: H
A32: H
A33: H
A34: H
A35: H
A36: H
A37: H
A38: H
A39: H
A40: H
A41: H
A42: H
A43: H
A44: H
A45: H
A46: H
A47: H
A48: H
A49: H
A50: H
A51: H
A52: L
A53: L
A54: L
A55: L
A56: L
A57: L
A58: L
A59: L
A60: L
A61: L
A62: H
A63: H
A64: H
A65: H
A66: H
A67: H
A68: H
A69: H
A70: H
A71: H
A72: H
A73: H
A74: H
A75: L
A76: L
A77: L
A78: L
A79: L
A80: L
A81: L
A82: L
A83: L
A84: L
A85: L
A86: H
A87: H
A88: H
A89: H
A90: H
A91: H
A92: H
A93: H
A94: H
A95: H
A96: H
A97: H
A98: H
A99: H
A100: H
A101: H
A102: H
A103: H
A104: H
A105: H
A106: H
A107: H
A108: H
A109: H
A110: H
A111: H
A112: H
A113: L
A114: L
A115: L
A116: L
A117: L
A118: L
A119: L
B23: L
B24: L
B25: L
B26: L
B27: H
B28: H
B29: H
B30: H
B31: H
B32: H
B33: H
B34: H
B35: H
B36: H
B37: H
B38: H
B39: H
B40: H
B41: H
B42: H
B43: H
B44: H
B45: H
B46: H
B47: H
B48: H
B49: H
B50: H
B51: H
B52: L
B53: L
B54: L
B55: L
B

Comparaison entre DSSP et notre projet

In [4]:
def get_dssp_secondary_structure(pdb_path):
    """
    Use DSSP via MDAnalysis to extract secondary structure from a PDB file.
    Maps each residue to one of three classes: H (helix), E (strand), L (loop).
    """
    u = mda.Universe(pdb_path)
    dssp = DSSP(u).run()

    dssp_dict = {}
    # Extract residue identifiers and chains
    resids = [res.resid for res in u.residues]
    chains = [res.segid.strip() or res.atoms[0].chainID for res in u.residues]
    sec_struct = dssp.results.dssp[0]  # DSSP returns a string of secondary structure labels

    # Convert DSSP symbols to simplified labels: H (helix), E (strand), L (loop/other)
    for resid, chain, ss in zip(resids, chains, sec_struct):
        label = ss if ss in ['H', 'E'] else 'L'
        dssp_dict[(chain, resid)] = label
    return dssp_dict


def get_prediction_dict(custom_result):
    """
    Convert custom prediction output (e.g., ['A12: H', 'A13: L']) into a dictionary.
    Keys are (chain, residue_id), values are secondary structure labels.
    """
    pred_dict = {}
    for entry in custom_result:
        chain_res, label = entry.split(':')
        chain = chain_res[0]  # First character is the chain
        resnum = int(chain_res[1:])  # Rest is the residue number
        pred_dict[(chain, resnum)] = label.strip()
    return pred_dict


def compare_structures(predicted, reference):
    """
    Compare predicted secondary structure against a reference (e.g. DSSP).
    Returns prediction accuracy as a percentage.
    """
    matched = 0
    total = 0

    # Compare labels for each residue in the reference
    for key, ref_ss in reference.items():
        if key in predicted:
            total += 1
            if predicted[key] == ref_ss:
                matched += 1

    accuracy = 100 * matched / total if total else 0
    return accuracy, matched, total


if __name__ == "__main__":
    pdb_path = "1f88.pdb"

    # Run custom secondary structure predictor
    residues = parse_pdb_residues(pdb_path)
    custom_result, _ = combine_secondary_structure(residues)
    predicted = get_prediction_dict(custom_result)

    # Run DSSP as reference standard
    reference = get_dssp_secondary_structure(pdb_path)

    # Compare custom prediction with DSSP results
    accuracy, matched, total = compare_structures(predicted, reference)

    print("\nComparison with DSSP:")
    print(f"  Residues compared: {total}")
    print(f"  Exact matches: {matched}")
    print(f"  Prediction accuracy: {accuracy:.2f}%")



Comparison with DSSP:
  Residues compared: 643
  Exact matches: 629
  Prediction accuracy: 97.82%


/home/gcluzeau/miniconda3/envs/env_proj_court/lib/python3.12/site-packages/MDAnalysis/analysis/base.py:542: UserWarning: Reader has no dt information, set to 1.0 ps
  self.times[idx] = ts.time
